
单阶段和二阶段目标检测方法模型表征图，
以及各模块主要有什么
one-stage和two-stage模型表征的异同
</div><i class="fa fa-lightbulb-o "></i>

In [ ]:
mmdetection/configs/base/default_runtime.py

checkpoint_config = dict(interval=1)# 每1个epoch存储一次模型
# yapf:disable
log_config = dict(
    interval=50,# 每50个batch输出一次信息
    hooks=[
        dict(type='TextLoggerHook'), # 控制台输出信息的风格
        # dict(type='TensorboardLoggerHook')
    ])
# yapf:enable
dist_params = dict(backend='nccl')# 分布式参数
log_level = 'INFO'#日志等级
load_from = None# 加载模型的路径，None表示从预训练模型加载，还可从代码更改为不加载
resume_from = None# 恢复训练模型的路径
workflow = [('train', 1)]

In [ ]:
mmdetection/configs/base/schedules/

schedule_1x.py，schedule_2x.py，schedule_20e.py
1x 表示12个epoch
2x 表示24个epoch
20e表示 20 个epochs ，通常 cascade模型中采用


# optimizer
optimizer = dict(type='SGD', lr=0.02, momentum=0.9, weight_decay=0.0001)# 优化参数，lr为学习率，momentum为动量因子，weight_decay为权重衰减因子
optimizer_config = dict(grad_clip=None)# 梯度均衡参数
# learning policy 学习率策略
lr_config = dict(
    policy='step',# 优化策略
    warmup='linear',# 初始的学习率增加的策略，linear为线性增加
    warmup_iters=500,# 在初始的500次迭代中学习率逐渐增加
    warmup_ratio=0.001,# 起始的学习率
    step=[8, 11])# 在第8和11个epoch时降低学习率
total_epochs = 12 # 最大epoch数

In [ ]:
# model settings


input_size = 300
model = dict(
    type='SingleStageDetector',# model类型
    pretrained='open-mmlab://vgg16_caffe',# 预训练模型：VGG16_caffe
    backbone=dict(
        type='SSDVGG',# backbone类型
        input_size=input_size,
        depth=16,
        with_last_pool=False,
        ceil_mode=True,
        out_indices=(3, 4),
        out_feature_indices=(22, 34),
        l2_norm_scale=20),
    neck=None,
    bbox_head=dict(
        type='SSDHead',
        in_channels=(512, 1024, 512, 256, 256, 256),#输入通道
        num_classes=80,
        anchor_generator=dict(
            type='SSDAnchorGenerator',
            scale_major=False,
            input_size=input_size,
            basesize_ratio_range=(0.15, 0.9),
            strides=[8, 16, 32, 64, 100, 300],# 在每个特征层对应于原图的步长
            ratios=[[2], [2, 3], [2, 3], [2, 3], [2], [2]]),#每个特征层对应先验框（prior box）的数量 是 [4,6,6,6,4,4]
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[.0, .0, .0, .0],
            target_stds=[0.1, 0.1, 0.2, 0.2])))
cudnn_benchmark = True
train_cfg = dict(
    assigner=dict(
        type='MaxIoUAssigner',
        pos_iou_thr=0.5,# 正样本的iou阈值
        neg_iou_thr=0.5,# 负样本的iou阈值
        min_pos_iou=0.,# 正样本的iou最小值。如果assign给ground truth的anchors中最大的IOU小于0，则忽略所有的anchors，否则保留最大IOU的anchor
        ignore_iof_thr=-1, #忽略bbox的阈值，当ground truth中包含需要忽略的bbox时使用，-1表示不忽略
        gt_max_assign_all=False),
    smoothl1_beta=1.,# 平滑L1系数
    allowed_border=-1, # 允许在bbox周围外扩一定的像素
    pos_weight=-1,
    neg_pos_ratio=3,
    debug=False)# debug模式
test_cfg = dict(
    nms=dict(type='nms', iou_thr=0.45),
    min_bbox_size=0,
    score_thr=0.02,
    max_per_img=200)


In [ ]:
data pipeline 

In [ ]:
_base_ = [
    '../_base_/models/ssd300.py', '../_base_/datasets/coco_detection.py',
    '../_base_/schedules/schedule_2x.py', '../_base_/default_runtime.py'
]
# dataset settings
dataset_type = 'CocoDataset' #数据集的类型
data_root = 'data/coco/'#数据集的根目录
# # 将输入图像norm，减去均值mean并除以方差std，to_rgb表示将bgr转为rgb

# 归一化再标准化的做法
# img = (img/255 - mean) / std

#只做标准化的做法
#img = (img - mean)/std


#  caffe 的img_norm方法是
#img_norm_cfg = dict(
#    mean=[103.530, 116.280, 123.675], std=[1.0, 1.0, 1.0], to_rgb=False)
# PyTorch 中经常看到的是下面这样
#img_norm_cfg = dict(
#    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)

img_norm_cfg = dict(mean=[123.675, 116.28, 103.53], std=[1, 1, 1], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile', to_float32=True),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='PhotoMetricDistortion',#PhotoMetricDistortion 数据增强，可以调整包括亮度，对比度，饱和度,色调
        brightness_delta=32,
        contrast_range=(0.5, 1.5),
        saturation_range=(0.5, 1.5),
        hue_delta=18),
    dict(
        type='Expand',
        mean=img_norm_cfg['mean'],
        to_rgb=img_norm_cfg['to_rgb'],
        ratio_range=(1, 4)),
    dict(
        type='MinIoURandomCrop',
        min_ious=(0.1, 0.3, 0.5, 0.7, 0.9),
        min_crop_size=0.3),
    dict(type='Resize', img_scale=(300, 300), keep_ratio=False),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels']),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(300, 300),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=False),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]
data = dict(
    samples_per_gpu=8,# 每个gpu分配的样本数量
    workers_per_gpu=3,# 每个gpu分配的线程数
    train=dict(
        _delete_=True,
        type='RepeatDataset',
        times=5,
        dataset=dict(
            type=dataset_type,
            ann_file=data_root + 'annotations/instances_train2017.json',
            img_prefix=data_root + 'train2017/',
            pipeline=train_pipeline)),
    val=dict(pipeline=test_pipeline),
    test=dict(pipeline=test_pipeline))
# optimizer
optimizer = dict(type='SGD', lr=2e-3, momentum=0.9, weight_decay=5e-4)
optimizer_config = dict(_delete_=True)


In [1]:
# %run mmdet/models/backbones/ssd_vgg.py


ImportError: attempted relative import with no known parent package